<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - KNN User User MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, user_knn as knn
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(neighbours,center):
    neighbours=int(neighbours)
    all_recs = []
    test_data = []
    algo_uu = knn.UserUser(neighbours,center=center,aggregate='weighted-average')
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
    return results["recip_rank"].mean()

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    neighbours = params['neighbours']
    center = params['center']
    metric = model_trainer(neighbours,center)
    print([neighbours,center,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space= {'neighbours': hp.uniform('neighbours', 1, 100),
       'center': hp.choice('center', ['True', 'False'])}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[42.462087705234666, 'False', 0.05416110220271075]                                                                     
,                                                                                                                      
[67.75893239584002, 'True', 0.05184344309009517]                                                                       
,                                                                                                                      
[13.433822720991602, 'True', 0.06016335823876753]                                                                      
,                                                                                                                      
[74.27349277506113, 'True', 0.04868002631222659]                                                                       
,                                                                                                                      
[52.9985855473087, 'False', 0.0546742689

KeyboardInterrupt: 

In [5]:
array = [[76.57444724550227, 'True', 0.04758463323232125]                       #145.42/81                                                
,                                                                                                                      
[9.521301756100101, 'True', 0.062328137172523584]                                                                      
,                                                                                                                      
[46.63059349946075, 'True', 0.05459118964273265]                                                                       
,                                                                                                                      
[75.61265810163842, 'True', 0.04754368808857482]                                                                       
,                                                                                                                      
[87.22508665636416, 'True', 0.0481127656254447]                                                                        
,                                                                                                                      
[58.91915608819299, 'True', 0.0509751027236648]                                                                        
,                                                                                                                      
[62.57556187556022, 'False', 0.04890979993391555]                                                                      
,                                                                                                                      
[60.41664591588658, 'False', 0.05070775073740352]                                                                      
,                                                                                                                      
[90.3764420726736, 'False', 0.04646606304365796]                                                                       
,                                                                                                                      
[37.086500570420924, 'False', 0.05327554974481623]                                                                     
,                                                                                                                      
[45.04824935274701, 'False', 0.05348249908866012]                                                                      
,                                                                                                                      
[88.1263676996598, 'True', 0.04794757102897059]                                                                        
,                                                                                                                      
[61.91744867669711, 'False', 0.0492403827093816]                                                                       
,                                                                                                                      
[8.48162719685299, 'False', 0.06408329649319042]                                                                       
,                                                                                                                      
[16.367038495745074, 'True', 0.060367350646050186]                                                                     
,                                                                                                                      
[70.5455778689412, 'False', 0.049629623336566356]                                                                      
,                                                                                                                      
[97.89460969981812, 'True', 0.04462907094167276]                                                                       
,                                                                                                                      
[27.81248984992698, 'True', 0.05553327325440082]                                                                       
,                                                                                                                      
[75.98516679778027, 'True', 0.04653711345957077]                                                                       
,                                                                                                                      
[48.70619893083006, 'True', 0.05098207714648514]                                                                       
,                                                                                                                      
[1.030572091503907, 'False', 0.04450795160351619]                                                                      
,                                                                                                                      
[3.929245434301258, 'False', 0.06854411368460116]                                                                      
,                                                                                                                      
[20.383393584246615, 'False', 0.05769537858739024]                                                                     
,                                                                                                                      
[2.270343658911563, 'False', 0.06212832169736424]                                                                      
,                                                                                                                      
[32.68264312510226, 'False', 0.054672452377022895]                                                                     
,                                                                                                                      
[10.195176888144891, 'False', 0.06250512366699879]                                                                     
,                                                                                                                      
[23.687557196086566, 'False', 0.062114621620844496]                                                                    
,                                                                                                                      
[5.376609959149197, 'False', 0.06709258695725065]                                                                      
,                                                                                                                      
[1.6246321943169768, 'False', 0.043322584218812264]                                                                    
,                                                                                                                      
[39.39313002640014, 'False', 0.055968418028079836]                                                                     
,                                                                                                                      
[16.547884737336872, 'False', 0.06036732247440973]                                                                     
,                                                                                                                      
[8.654866954385785, 'False', 0.06923446764775741]                                                                      
,                                                                                                                      
[12.146323321136556, 'False', 0.06410531152494467]                                                                     
,                                                                                                                      
[31.25318179147599, 'False', 0.056127018427814514]                                                                     
,                                                                                                                      
[24.155788759923045, 'False', 0.056290120219499994]                                                                    
,                                                                                                                      
[15.109719333516635, 'False', 0.06836777459254284]                                                                     
,                                                                                                                      
[40.548614549847215, 'False', 0.053837077192136686]                                                                    
,                                                                                                                      
[55.50861794028374, 'False', 0.048263094906725694]                                                                     
,                                                                                                                      
[5.847588185128103, 'False', 0.06977810057630561]                                                                      
,                                                                                                                      
[34.8090261250633, 'False', 0.0576769925702202]                                                                        
,                                                                                                                      
[68.01777222931361, 'True', 0.04994835673350556]                                                                       
,                                                                                                                      
[52.318524843256675, 'False', 0.049515350160245476]                                                                    
,                                                                                                                      
[80.57259379646338, 'False', 0.047798836518733796]                                                                     
,                                                                                                                      
[43.125756693001065, 'True', 0.054492836926940555]                                                                     
,                                                                                                                      
[20.579727994335485, 'False', 0.057124167948861757]                                                                    
,                                                                                                                      
[5.55583018163987, 'False', 0.06773700395542957]                                                                       
,                                                                                                                      
[29.173894433100596, 'False', 0.058553985666197336]                                                                    
,                                                                                                                      
[12.924158881202876, 'True', 0.060639308990319093]                                                                     
,                                                                                                                      
[7.969058785536952, 'False', 0.07009572216377619]                                                                      
,                                                                                                                      
[99.56795494420574, 'True', 0.044209427547732556]                                                                      
,                                                                                                                      
[94.58258979993754, 'False', 0.04691575043122885]                                                                      
,                                                                                                                      
[83.24527926259063, 'False', 0.04776897153857065]                                                                      
,                                                                                                                      
[66.76293960531561, 'True', 0.04780715188078541]                                                                       
,                                                                                                                      
[19.13100484115458, 'False', 0.06252594098088947]                                                                      
,                                                                                                                      
[25.867815927269046, 'False', 0.06057295226911927]                                                                     
,                                                                                                                      
[47.73404562170202, 'True', 0.05186037091625072]                                                                       
,                                                                                                                      
[35.107356148283266, 'False', 0.054341889474483436]                                                                    
,                                                                                                                      
[56.38406643444546, 'False', 0.054987198831493916]                                                                     
,                                                                                                                      
[6.905918250206244, 'False', 0.07003769387479904]                                                                      
,                                                                                                                      
[52.13430585587166, 'True', 0.05266812217309767]                                                                       
,                                                                                                                      
[72.33089341507136, 'False', 0.04952991231442287]                                                                      
,                                                                                                                      
[18.305530177349823, 'False', 0.05952932806251704]                                                                     
,                                                                                                                      
[21.60605770370185, 'False', 0.05812910803108274]                                                                      
,                                                                                                                      
[43.77587853636253, 'True', 0.05363559962274609]                                                                       
,                                                                                                                      
[12.364787487334958, 'False', 0.0633485415312651]                                                                      
,                                                                                                                      
[6.929750119688222, 'False', 0.07222258811592182]                                                                      
,                                                                                                                      
[7.66469977614954, 'False', 0.07436257397872661]                                                                       
,                                                                                                                      
[9.764415309631737, 'False', 0.06533896631107931]                                                                      
,                                                                                                                      
[2.5214635314189886, 'False', 0.0594710171450353]                                                                      
,                                                                                                                      
[15.190115156855976, 'False', 0.0613630165360487]                                                                      
,                                                                                                                      
[28.22634952579421, 'False', 0.05576196850572142]                                                                      
,                                                                                                                      
[24.587396290475265, 'False', 0.056879863080796726]                                                                    
,                                                                                                                      
[3.4306540264731216, 'False', 0.06552165592104003]                                                                     
,                                                                                                                      
[31.82987536521334, 'False', 0.05525989531288185]                                                                      
,                                                                                                                      
[7.9781411748454385, 'False', 0.06858722954161607]                                                                     
,                                                                                                                      
[37.68431563362565, 'False', 0.05625858172820415]                                                                      
,                                                                                                                      
[11.606414033016463, 'False', 0.06208289742730461]                                                                     
,                                                                                                                      
[22.424214639432165, 'False', 0.06163515943481068]                                                                     
,                                                                                                                      
[17.043290921358455, 'False', 0.06178913351091268]                                                                     
,                                                                                                                      
[26.40524240056728, 'False', 0.055616787457369615]                                                                     
,                                                                                                                      
[14.164138535642476, 'False', 0.06154667238583589],
[18.108936203163644, 'True', 0.057865280605864974]                                                                     
,                                                                                                                      
[51.51904044368786, 'True', 0.0544692498219766]                                                                        
,                                                                                                                      
[65.85448586935298, 'False', 0.04939631166406828]                                                                      
,                                                                                                                      
[26.986374726315145, 'True', 0.058440079107632305]                                                                     
,                                                                                                                      
[23.43716687869319, 'False', 0.059404193273129215]                                                                     
,                                                                                                                      
[76.10936679843513, 'False', 0.048854043406152625]                                                                     
,                                                                                                                      
[47.945540100941514, 'True', 0.052443900250827964]                                                                     
,                                                                                                                      
[44.43192356710524, 'False', 0.05205917521906408]                                                                      
,                                                                                                                      
[16.71350662859225, 'True', 0.060624758434676115]                                                                      
,                                                                                                                      
[86.21100463538554, 'False', 0.04623185692086939]                                                                      
,                                                                                                                      
[81.06399150030626, 'True', 0.049343861957066235]                                                                      
,                                                                                                                      
[16.06241437433974, 'True', 0.05892991790839103]
,
[42.462087705234666, 'False', 0.05416110220271075]                                                                     
,                                                                                                                      
[67.75893239584002, 'True', 0.05184344309009517]                                                                       
,                                                                                                                      
[13.433822720991602, 'True', 0.06016335823876753]                                                                      
,                                                                                                                      
[74.27349277506113, 'True', 0.04868002631222659]                                                                       
,                                                                                                                      
[52.9985855473087, 'False', 0.05467426890407397]                                                                       
,                                                                                                                      
[13.92730682311614, 'True', 0.06501804497429185]                                                                       
,                                                                                                                      
[30.85534366352042, 'False', 0.059103806374222946]                                                                     
]

In [6]:
metric = pd.DataFrame(array)
metric.columns = ['neighbours','center','MRR']
metric

,neighbours,center,MRR
0,76.574447,True,0.047585
1,9.521302,True,0.062328
2,46.630593,True,0.054591
3,75.612658,True,0.047544
4,87.225087,True,0.048113
...,...,...,...
95,13.433823,True,0.060163
96,74.273493,True,0.048680
97,52.998586,False,0.054674
98,13.927307,True,0.065018


In [7]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,neighbours,center,MRR
66,7.664700,False,0.074363
65,6.929750,False,0.072223
48,7.969059,False,0.070096
58,6.905918,False,0.070038
38,5.847588,False,0.069778
31,8.654867,False,0.069234
74,7.978141,False,0.068587
21,3.929245,False,0.068544
35,15.109719,False,0.068368
45,5.555830,False,0.067737


In [8]:
metric.to_csv('metric_MRR.csv', index=False)